In [1]:
#using snake casing to name variables and functions

import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import csv
import re
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from openpyxl import load_workbook
from datetime import datetime

In [2]:
#global variables
city_name = "hyderabad"
zomato_url = "https://www.zomato.com"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}


In [3]:
def get_page_content(link,headers):
    try:
        #link = 'https://www.zomato.com/hyderabad/antera-kitchen-and-bar-jubilee-hills'
        response = requests.get(link, headers=headers)
        #response = requests.get("https://www.zomato.com/hyderabad/papadams-blue-kothapet/order")
        return {'status':response.status_code,
                'content': response.content}
    
    except requests.exceptions.HTTPError as errh:
        return {'status':response.status_code,
                'content':"Http Error:"+errh}
    except requests.exceptions.ConnectionError as errc:
        return {'status':response.status_code,
                'content':"Error Connecting:"+errc}
    except requests.exceptions.Timeout as errt:
        return {'status':response.status_code,
                'content':"Timeout Error:"+errt}
    except requests.exceptions.RequestException as err:
        return {'status':response.status_code,
                'content':"OOps: Something Else"+err}
    

In [4]:
def get_restaurant_links_via_html(html):
    try:
        if(html):
            soup = BeautifulSoup(html,"lxml")
            search_list = soup.find_all("div", {"class":"jumbo-tracker"})
#             print("number of search_list:"+str(len(search_list)))

            restaurant_links=[]
            ################################ 
            #need to edit here, indices starts from 22 to len() ??? we need all!
            ################################ 
            for i in range(22,len(search_list)):
                link = search_list[i].div.find('a')['href']
                link = link.replace('/order', '')
#                 print(link)
                restaurant_links.append(zomato_url+link)

#             print("total number of restaurant links:" + str(len(restaurant_links)))
            
            #     for x in (details[0].find_all(True)):
            #         if(x.string!=None):
            #             print(x.string+' {}'.format(i))
            #             i+=1
            e = {
                'status':200,
                'content':restaurant_links
            }
#             print(e)
            return e
    except Exception as e:
        e = {"status":999,'error':str(e)}
        print(e)
        return e

# print(get_restaurant_links(city_name))

In [5]:
def get_restaurant_links_via_href(city_name):
    try:
        url = "https://www.zomato.com/{}/restaurants".format(city_name)
#         print(url)
        response = get_page_content(url,headers)
#         print(type(response))
        if(response['status']==200):
            content = response['content']
            soup = BeautifulSoup(content,"lxml")
            search_list = soup.find_all("div", {"class":"jumbo-tracker"})
#             print("number of search_list:"+str(len(search_list)))

            restaurant_links=[]
            ################################ 
            #need to edit here, indices starts from 22 to len() ??? we need all!
            ################################ 
            for i in range(22,len(search_list)):
                link = search_list[i].div.find('a')['href']
                link = link.replace('/order', '')
#                 print(link)
                restaurant_links.append(zomato_url+link)

#             print("total number of restaurant links:" + str(len(restaurant_links)))
            
            #     for x in (details[0].find_all(True)):
            #         if(x.string!=None):
            #             print(x.string+' {}'.format(i))
            #             i+=1
            status = {
                'status':200,
                'content':restaurant_links
            }
#             print(e)
            return status
    except Exception as e:
        e = {"status":999,'error':str(e)}
        print(e)
        return e

# print(get_restaurant_links(city_name))

In [6]:
# #just learnt abt beautiful-soup-4!

# jumbo_tracker = soup.select('.jumbo-tracker')
# tag = jumbo_tracker[0]
# #print tag-name
# print(tag.name) # yes its a div tag!
# #u can edit tag name in the current soup too!
# print(tag.attrs)
# print(tag.get('class'))
# print(type(soup))
# print(len(soup.find_all(attrs={'class':'jumbo-tracker'})))
# print(len(soup.find_all('div',attrs={'class':'jumbo-tracker'})))
# print(len(soup.select('div[class="jumbo-tracker"]')))
# print(soup.select('div[class="jumbo-tracker"]')[-1])
# print()

In [7]:
#returns get restaurant page content for 200 successful response

def get_primary_data(content):
    try:
        
        soup = BeautifulSoup(content,"lxml")

        main = soup.find("main").div

        # location = soup.find("div",{"aria-label":"Breadcrumb"}).find_all("a")
        location = main.section.div.find_all("a")
#         print(location)

        loc_tags = [tag["title"] for tag in location]
        
        rest_all_tags = {'country':'','city':'','city_2':'','region':'','restaurant_name':''}
        if(len(loc_tags)>=1):rest_all_tags ["country"] = loc_tags[1]
        if(len(loc_tags)>=2):rest_all_tags ["city"] = loc_tags[2]
        if(len(loc_tags)>=3):rest_all_tags ["city_2"] = loc_tags[3]
        if(len(loc_tags)>=4):rest_all_tags ["region"] = loc_tags[4]
        if(len(loc_tags)>=5):rest_all_tags ["restaurant_name"] = loc_tags[5]
#         print(len(loc_tags))
#         print(loc_tags)
    #rest_all_tags["order_online"] = True if(loc_tags[6]=='Order Online') else False
    # order_online is invalid in off timings

    # loc_tags = loc_tags[1:len(loc_tags)-1]

#         print(rest_all_tags)
        
        e = {
            'status': 200,
            'content': rest_all_tags
        }
#         print(e)
        return e
    
    except Exception as e:
        e =  {
            'status': 998,
            'error': str(e)
        }
        print(e)
        return e

In [8]:
def get_secondary_data(content):
    try:
        soup = BeautifulSoup(content,"lxml")
        main = soup.find("main").div

        stuff = main.section.next_sibling.next_sibling.section
        stuff = stuff.get_text("!@#$%^&",strip=True).split("!@#$%^&")
    #         print(stuff)

        rest_all_tags={'restaurant_name_2':'','dining_rating':0.0,'dining_reviews':0,'delivery_rating':0.0,'delivery_reviews':0,'cuisines':[],'location':''}
        if(len(stuff)>=0):rest_all_tags["restaurant_name_2"] = stuff[0]
        if(len(stuff)>=1):rest_all_tags["dining_rating"] = stuff[1]
        if(len(stuff)>=3):rest_all_tags["dining_reviews"] = stuff[3]
        if(len(stuff)>=5):rest_all_tags["delivery_rating"] = stuff[5]
        if(len(stuff)>=7):rest_all_tags["delivery_reviews"] = stuff[7]
        if(len(stuff)>=9):rest_all_tags["cuisines"]=[stuff[9]]

        for i in range(10,len(stuff)):
            if(stuff[i-1]==','):
                rest_all_tags["cuisines"].append(stuff[i])

        cur = 9+(len(rest_all_tags["cuisines"]*2)-1)
        if(len(stuff)>=cur):rest_all_tags["location"] = stuff[cur]

        #timings can't be captured in night or off timings!

    #         print(len(stuff))
    #         print(stuff)

    #         print(rest_all_tags)
        e = {
            'status': 200,
            'content':rest_all_tags
        }
    #         print(e)
        return e
    except Exception as e:
        e = {
            'status':997,
            'error':str(e)
        }
        print(e)
        print(stuff)
        return e
    

In [9]:
'''
for the rest of the info,
just get to the last but one sibling and find text,
then find its class then traverse into new one,
like cuisines: a,b,c
average_cost: a,b,c
'''

#
def get_more_info(content):
    try:
        soup = BeautifulSoup(content,"lxml")
        main = soup.find("main").div
        
        #another way for cuisines:
        
        # print(stuff.get_text('!@#$%^&').split("!@#$%^&"))
        # print(len(stuff.get_text('!@#$%^&').split("!@#$%^&")))

        search_elements = ['Cuisines', 'Popular Dishes', '₹','More Info']
        # print(stuff)
        stuff = main.section
        for i in range(4): #iterating through main.section siblings!
             stuff = stuff.next_sibling
        
        stuff = stuff.section.section.article.section.div.next_siblings

        cuisines = []
        popular_dishes = []
        average_costs = []
        more_info = []
        
        for q in stuff:
        #     print(q)
            if q.text == search_elements[0]:
                cuisines = q.next_sibling.get_text("!@#$%^&").split("!@#$%^&")
            elif q.text == search_elements[1]:
                popular_dishes = q.next_sibling.get_text("!@#$%^&").split("!@#$%^&")
            elif search_elements[2] in q.text:
                average_costs.append(q.text)
            elif q.text == search_elements[3]:
                more_info = q.next_sibling.get_text("!@#$%^&").split("!@#$%^&")


#         print(cuisines)
#         print(popular_dishes)
#         print(average_costs)
#         print(more_info)

        rest_all_tags={'cuisines_2':[],'popular_dishes':[],'average_costs':[],'more_info':[]}
        if(cuisines):rest_all_tags['cuisines_2'] = cuisines
        if(popular_dishes):rest_all_tags['popular_dishes'] = popular_dishes
        if(average_costs):rest_all_tags['average_costs']=average_costs
        if(more_info):rest_all_tags['more_info']=more_info
#         print(f"rest all tags:{rest_all_tags}")
        e = {
            'status': 200,
            'content':rest_all_tags
        }
#         print(e)
        return e
    except Exception as e:
        e = {
            'status': 996,
            'content':str(e)
        }
        print(e)
        return e

In [10]:
def scroll_down(link):
    SCROLL_PAUSE_TIME = 3
    browser=webdriver.Chrome()
#     browser.set_script_timeout(150000) #150 session or browser timeout!
    browser.get(link)
    while True:
        prev_height = browser.execute_script("return document.body.scrollHeight")
#         print(prev_height)
        ActionChains(browser).send_keys(Keys.END).send_keys(Keys.PAGE_UP).perform()
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = browser.execute_script("return document.body.scrollHeight")
#         print(new_height)
        if new_height == prev_height:
            html = browser.page_source
            break
#     while True:
#         prev_height = browser.execute_script("return document.body.scrollHeight")
#         print(prev_height)
#         browser.execute_script("window.scrollTo(0,document.body.scrollHeight/2)")#height/2 last time
#         time.sleep(SCROLL_PAUSE_TIME)
#         new_height = browser.execute_script("return document.body.scrollHeight")
#         print(new_height)
# #         print(html)
#         if new_height == prev_height:
#             break
#     while True:
#         prev_height = browser.execute_script("return document.body.scrollHeight")
#         print(prev_height)
#         browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
#         time.sleep(SCROLL_PAUSE_TIME)
#         new_height = browser.execute_script("return document.body.scrollHeight")
#         print(new_height)
        
#         print(html)
#         if new_height == prev_height:
#             html = browser.page_source
#             break
    browser.close()
    return html

In [11]:
# demo = [{
#     'aaa':1,
#     'bbb':2,
#     'ccc':3
# }]
# demodf = pd.DataFrame(demo)
# # demodf.to_excel("{}_restaurants_details.xlsx".format("demo"), index=False)
# # demodf.to_excel("{}_restaurants_details.xlsx".format("demo"), index=False)
# # print(demodf)
# demo2 = [{
#     'aaa':4,
#     'bbb':5,
#     'ccc':6
# }]
# demo2df = pd.DataFrame(demo2)

# print(demodf)


# #chat gpt
# file_path = "{}_restaurants_details.xlsx".format("demo3")
# excel_writer = pd.ExcelWriter(file_path, engine='openpyxl',mode='a')
# book = load_workbook(file_path)
# book = excel_writer.book
# sheet_name = book.sheetnames[0]
# if sheet_name in book.sheetnames:
#     # Load existing sheet
#     excel_writer.sheets = {ws.title: ws for ws in book.worksheets}
#     print(excel_writer.sheets)
# else:
#     # Create new sheet
# #     demodf = pd.DataFrame()
#     demodf.to_excel(excel_writer, index=False, header=False, sheet_name=sheet_name)
# # new_dataframe = pd.DataFrame()  # Replace ... with your new data
# print(demo2df)
# demo2df.to_excel(excel_writer, index=False, header=False, sheet_name=sheet_name, startrow=excel_writer.sheets[sheet_name].max_row)
# excel_writer.save()
# excel_writer.close()


In [12]:
def append_into_excel(file_path, data):
    
#     file_path = "{}_restaurants_details.xlsx".format("demo")

# Load existing data from the Excel file
    existing_data = pd.read_excel(file_path)

# Create or retrieve the new dataframe you want to append
# new_dataframe = pd.DataFrame(...)  # Replace ... with your new data

# Concatenate the existing data and the new dataframe
    concatenated_data = pd.concat([existing_data, data], ignore_index=True)

# Write the concatenated data to the Excel file
    with pd.ExcelWriter(file_path, engine='xlsxwriter', mode='w') as writer:
        concatenated_data.to_excel(writer, index=False, header=True)
#     print("Data appended successfully.")
    
    
    

# demo = [{
#     'aaa':1,
#     'bbb':2,
#     'ccc':3
# }]
# demodf = pd.DataFrame(demo)
# demodf.to_excel("{}_restaurants_details.xlsx".format("demo"), index=False)
# demodf.to_excel("{}_restaurants_details.xlsx".format("demo"), index=False)
# print(demodf)
# demo2 = [{
#     'aaa':4,
#     'bbb':5,
#     'ccc':6
# }]
# demo2df = pd.DataFrame(demo2)
# path = create_excel_file('demo2_file',demo2df.columns)
# append_into_excel(path,demodf)
# print(demodf)
# append_into_excel(path,demo2df)

In [13]:
def create_excel_file(file_name,columns):
    data = pd.DataFrame(columns=columns)
    now = datetime.now()
    file_name = file_name + now.strftime("%d.%m.%Y %H-%M-%S ")
    file_path = "restaurant_details/{}-details.xlsx".format(file_name)
    data.to_excel(file_path, index=False)
    return file_path
        

In [14]:
def scraper():
    city_links_hyderabad = ['https://www.zomato.com/hyderabad/jubilee-hills-restaurants','https://www.zomato.com/hyderabad/gachibowli-restaurants','https://www.zomato.com/hyderabad/banjara-hills-restaurants','https://www.zomato.com/hyderabad/hitech-city-restaurants','https://www.zomato.com/hyderabad/madhapur-restaurants','https://www.zomato.com/hyderabad/kondapur-restaurants','https://www.zomato.com/hyderabad/kukatpally-restaurants','https://www.zomato.com/hyderabad/begumpet-restaurants','https://www.zomato.com/hyderabad/himayath-nagar-restaurants','https://www.zomato.com/hyderabad/tolichowki-restaurants','https://www.zomato.com/hyderabad/ameerpet-restaurants','https://www.zomato.com/hyderabad/somajiguda-restaurants','https://www.zomato.com/hyderabad/film-nagar-restaurants','https://www.zomato.com/hyderabad/paradise-circle-restaurants','https://www.zomato.com/hyderabad/sainikpuri-restaurants','https://www.zomato.com/hyderabad/necklace-road-restaurants','https://www.zomato.com/hyderabad/kothapet-restaurants','https://www.zomato.com/hyderabad/sd-road-restaurants','https://www.zomato.com/hyderabad/abids-restaurants','https://www.zomato.com/hyderabad/kompally-restaurants','https://www.zomato.com/hyderabad/masab-tank-restaurants','https://www.zomato.com/hyderabad/gandipet-restaurants','https://www.zomato.com/hyderabad/lb-nagar-restaurants','https://www.zomato.com/hyderabad/miyapur-restaurants','https://www.zomato.com/hyderabad/karkhana-restaurants','https://www.zomato.com/hyderabad/basheer-bagh-restaurants','https://www.zomato.com/hyderabad/panjagutta-restaurants','https://www.zomato.com/hyderabad/as-rao-nagar-restaurants','https://www.zomato.com/hyderabad/uppal-restaurants']
#     city_links_hyderabad.reverse()
#     city_links_hyderabad = [city_links_hyderabad[-1]];
    city_names = []
    for city_link in city_links_hyderabad:
#             link = "https://www.zomato.com/hyderabad/{}-restaurants".format(city_name)
#         link = "https://www.zomato.com/hyderabad/jubilee-hills-restaurants"
#         print(city_link)
        html = scroll_down(city_link)
#         print("html is working?")
        r = get_restaurant_links_via_html(html)
#         print("number of search_list:"+str(len(search_list)))
#         r = get_restaurant_links_via_href("hyderabad")
#         print("r value!")
        restaurant_links=[]
        if(r['status']!=200):
            html = scroll_down(link)
            r=get_restaurant_links_via_html(html)
        else:
            restaurant_links = r['content']

        columns = ['country','city','city_2','region','restaurant_name','restaurant_name_2','dining_rating', 'dining_reviews', 'delivery_rating', 'delivery_reviews','cuisines','location', 'cuisines_2', 'popular_dishes','average_costs','more_info']

        #creating an excel file to append into this later
        file_path = create_excel_file(city_link.split('/')[-1],columns)
        city_names.append(city_link.split('/')[-1])

        for link in restaurant_links:
            details={}
            page = get_page_content(link, headers)
    #         if(page['status'] != 200):
    #             continue
            page = page['content']
    #         print(link)
    #         print(details)

            data = get_primary_data(page)
            if(data['status']==200):
                details.update(data['content'])
            else:
                details.update({'country':'','city':'','city_2':'','region':'','restaurant_name':''})
    #         print(link)
    #         print(details)


            data = get_secondary_data(page)
            if(data['status']==200):
                details.update(data['content'])
            else:
                details.update({'restaurant_name_2':'','dining_rating':0.0,'dining_reviews':0,'delivery_rating':0.0,'delivery_reviews':0,'cuisines':[],'location':''})
    #         print(link)
    #         print(details)

            data = get_more_info(page)
            if(data['status']==200):
                details.update(data['content'])
            else:
                details.update({'cuisines_2':[],'popular_dishes':[],'average_costs':[],'more_info':[]})

            # appending restaurant details
#             print(details)
#             print(columns)
            details = [details]
            details_df=pd.DataFrame(details,columns=columns)
#             details_df = pd.DataFrame.from_dict(details, orient='columns',columns=columns)
#             details_df.transpose()
#             print(details_df)

            append_into_excel(file_path,details_df)


#                 df = pd.DataFrame(technologies)
#                 new_row = {'Courses':'Hyperion', 'Fee':24000, 'Duration':'55days', 'Discount':1800}
#                 df2 = df.append(new_row, ignore_index=True)

#                 all_restaurant_details = all_restaurant_details.append(details, ignore_index=True)
    #         print(link)
    #         print(details)
        
        print("#"*20)
        print(f"city link: {city_link}")
        print(f"number of restaurants: {len(restaurant_links)}")
        print("data appended successfully!")
        print("#"*20)


#         df = pd.DataFrame(all_restaurant_details)
#         print(df)
#         print(all_restaurant_details)
#             print("all_restaurants_details: {}".format(len(all_restaurant_details)))

        #to excel file
#         all_restaurant_details.to_excel("{}_restaurants_details.xlsx".format("jubilee_hills"), index=False)


    
    
    
    
    
    

####################
city link: https://www.zomato.com/hyderabad/uppal-restaurants
number of restaurants: 72
data appended successfully!
####################


In [15]:
# city_names=[]
# for city_link in city_links_hyderabad:
#     city_names.append(city_link.split('/')[-1])
# main_file = "hyderabad-compiled-restaurants-details.xlsx"
# create_excel_file(main_file,columns)

# main_excel_df = pd.read_excel(main_file)
# with pd.ExcelWriter(file_path, engine='xlsxwriter', mode='w') as writer:
#     concatenated_data.to_excel(writer, index=False, header=True)
# print("Data appended successfully.")
# print(city_names)
# file_name = ""
# now = datetime.now()
# file_name = file_name + now.strftime("%d.%m.%Y %H-%M-%S")
# print(type(file_name))
# print(file_name)

In [16]:
# this_link = "https://www.zomato.com/hyderabad/ram-ki-bandi-gachibowli"
# random_html = get_page_content(this_link,headers)
# get_secondary_data(random_html['content'])